# Random Forest

Leitura dos datasets tratados

In [1]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e Modelação

In [2]:
label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])

In [3]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=183)

params =  {'n_estimators': [10, 50, 100], 
           'max_depth': list(range(1,20)), 
           'criterion':['gini','entropy']}

grid = GridSearchCV(estimator=rf,
                    param_grid=params,
                    cv=5,
                    n_jobs=1,
                    verbose=2)
grid.fit(X_treino, y_treino)

grid.best_score_, grid.best_params_

Fitting 5 folds for each of 114 candidates, totalling 570 fits
[CV] END .......criterion=gini, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END .......criterion=gini, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END .......criterion=gini, max_depth=1, n_estimators=10; total time=   0.1s
[CV] END .......criterion=gini, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END .......criterion=gini, max_depth=1, n_estimators=10; total time=   0.1s
[CV] END .......criterion=gini, max_depth=1, n_estimators=50; total time=   0.2s
[CV] END .......criterion=gini, max_depth=1, n_estimators=50; total time=   0.2s
[CV] END .......criterion=gini, max_depth=1, n_estimators=50; total time=   0.3s
[CV] END .......criterion=gini, max_depth=1, n_estimators=50; total time=   0.2s
[CV] END .......criterion=gini, max_depth=1, n_estimators=50; total time=   0.2s
[CV] END ......criterion=gini, max_depth=1, n_estimators=100; total time=   0.4s
[CV] END ......criterion=gini, max_depth=1, n_

(0.8307025644785655,
 {'criterion': 'entropy', 'max_depth': 7, 'n_estimators': 10})

In [5]:
predictions = grid.predict(X_teste)

predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1
df.index.name = 'RowId'

df.to_csv(f'predictions/randomForestPrediction.csv')